##### Importamos las librerias Necesarias

In [342]:
import pandas as pd
import re
import numpy as np
from datetime import datetime

##### Creamos el Dataset con el archivo movies_dataset.csv

In [343]:
df = pd.read_csv('Dataset/movies_dataset.csv')

/var/folders/mm/qj2l_7611k7cdqq50jvw32nc0000gn/T/ipykernel_2418/2393210234.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Dataset/movies_dataset.csv')


In [344]:
df = df.drop(df[df['id'] == '1997-08-20'].index)
df = df.drop(df[df['id'] == '2012-09-29'].index)
df = df.drop(df[df['id'] == '2014-01-01'].index)

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [345]:
# Definir el patrón de expresión regular
patron = r"'name': '(.*?)',"

# Función para extraer el nombre usando expresiones regulares
def extraer_nombre_coleccion(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.search(patron, valor)
    if resultado:
        return resultado.group(1)
    else:
        return ""

# Aplicar la función a toda la columna 'belongs_to_collection' y asignar los resultados a la misma columna
df['belongs_to_collection'] = df['belongs_to_collection'].apply(extraer_nombre_coleccion)

# Imprimir el DataFrame resultante
df['belongs_to_collection'].head()

0              Toy Story Collection
1                                  
2         Grumpy Old Men Collection
3                                  
4    Father of the Bride Collection
Name: belongs_to_collection, dtype: object

In [346]:
patron = r"'name': '(.*?)'},"

def extraer_nombres_generos(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['genres'] = df['genres'].apply(extraer_nombres_generos)

# Imprimir el DataFrame resultante
df['genres'].head()

0     [Animation, Comedy]
1    [Adventure, Fantasy]
2               [Romance]
3         [Comedy, Drama]
4                        
Name: genres, dtype: object

In [347]:
patron = r"'name': '(.*?)',"

def extraer_nombres_companies(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['production_companies'] = df['production_companies'].apply(extraer_nombres_companies)
# Imprimir el DataFrame resultante
df['production_companies']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45461                                                     
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                     
Name: production_companies, Length: 45463, dtype: object

In [348]:
patron = r"name': '(.*?)'}"

def extraer_nombres_countries(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['production_countries'] = df['production_countries'].apply(extraer_nombres_countries)

# Imprimir el DataFrame resultante
df['production_countries']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45461                        [Iran]
45462                 [Philippines]
45463    [United States of America]
45464                      [Russia]
45465              [United Kingdom]
Name: production_countries, Length: 45463, dtype: object

In [349]:
patron = r"name': '(.*?)'}"

def extraer_nombres_languajes(valor):
    if pd.isnull(valor):
        return ''
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ''

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['spoken_languages'] = df['spoken_languages'].apply(extraer_nombres_languajes)

# Imprimir el DataFrame resultante
df['spoken_languages']

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45461                [فارسی]
45462                     []
45463              [English]
45464                       
45465              [English]
Name: spoken_languages, Length: 45463, dtype: object

2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [350]:
num_nulos_budget = df['budget'].isnull().sum()
num_nulos_revenue = df['revenue'].isnull().sum()
print("nulos budget:",num_nulos_budget,"nulos revenue:", num_nulos_revenue)

nulos budget: 0 nulos revenue: 3


In [351]:
df['budget'] = df['budget'].fillna(0)
df['revenue'] = df['revenue'].fillna(0)

In [352]:
num_nulos_budget = df['budget'].isnull().sum()
num_nulos_revenue = df['revenue'].isnull().sum()
print("nulos budget:",num_nulos_budget,"nulos revenue:", num_nulos_revenue)

nulos budget: 0 nulos revenue: 0


3. Los valores nulos del campo release date deben eliminarse.

In [353]:
num_nulos_release_date = df['release_date'].isnull().sum()
num_nulos_release_date

87

In [354]:
df= df.dropna(subset=['release_date'])

In [355]:
num_nulos_release_date = df['release_date'].isnull().sum()
num_nulos_release_date

0

4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [356]:
# Definir función para convertir la fecha en formato de cadena
def voltear_fecha(fecha_str):
    try:
        fecha_obj = datetime.strptime(fecha_str, "%d-%m-%Y")
        return fecha_obj.strftime("%Y-%m-%d")
    except ValueError:
        return fecha_str

# Aplicar la función a toda la columna 'Fecha'
df['release_date'] = df['release_date'].apply(voltear_fecha)

# Imprimir el DataFrame resultante
df['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45460    1991-05-13
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45376, dtype: object

5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [357]:
# Definir función para calcular el retorno
def calcular_retorno(row):
    if float(row['revenue']) == 0 or float(row['budget']) == 0:
        return 0
    else:
        return float(row['revenue']) / float(row['budget'])
    
df['return'] = df.apply(calcular_retorno, axis=1)

6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [358]:
df = df.drop('video', axis=1)
df = df.drop('imdb_id', axis=1)
df = df.drop('adult', axis=1)
df = df.drop('original_title', axis=1)
df = df.drop('poster_path', axis=1)
df = df.drop('homepage', axis=1)

Agregamos la columna de los directores desde el archivo csv de credits.

In [370]:
df_credits = pd.read_csv('Dataset/credits.csv')
patron = r"'Director', 'name': '(.*?)', "

def extraer_nombres_director(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

df['id'] = df['id'].astype(int)
df_merged = pd.merge(df, df_credits, on='id')
df_merged['director'] = df_merged['crew'].apply(extraer_nombres_director)

print(df_merged['director'])

0           [John Lasseter]
1            [Joe Johnston]
2           [Howard Deutch]
3         [Forest Whitaker]
4           [Charles Shyer]
                ...        
45446          [John Irvin]
45447            [Lav Diaz]
45448      [Mark L. Lester]
45449    [Yakov Protazanov]
45450       [Daisy Asquith]
Name: director, Length: 45451, dtype: object


In [374]:
print(df_merged.at[189, 'director'])

In [376]:
df_merged = df_merged.drop('cast', axis=1)
df_merged = df_merged.drop('crew', axis=1)

In [361]:
df_merged

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,director
0,Toy Story Collection,30000000,"[Animation, Comedy]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,12.451801,[John Lasseter]
1,,65000000,"[Adventure, Fantasy]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,[Joe Johnston]
2,Grumpy Old Men Collection,0,[Romance],15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,[Howard Deutch]
3,,16000000,"[Comedy, Drama]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,[Forest Whitaker]
4,Father of the Bride Collection,0,,11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,,0,"[Drama, Action]",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,0.0,104.0,[English],Released,NaN,Robin Hood,5.7,26.0,0.000000,[John Irvin]
45447,,0,,111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,0.0,360.0,[],Released,NaN,Century of Birthing,9.0,3.0,0.000000,[Lav Diaz]
45448,,0,"[Action, Drama]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,0.0,90.0,[English],Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,[Mark L. Lester]
45449,,0,,227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,0.0,87.0,,Released,NaN,Satan Triumphant,0.0,0.0,0.000000,[Yakov Protazanov]


In [362]:
df_merged['original_language'] = df['original_language'].fillna('')
df_merged['overview'] = df['overview'].fillna('')
df_merged['production_countries'] = df['production_countries'].fillna('vacio')
df_merged['original_language'] = df_merged['original_language'].fillna('')

In [363]:
df_merged.to_csv('df_limpio.csv', index=False)

In [364]:
df_pruebas = pd.read_csv('df_limpio.csv')

def peliculas_idioma( Idioma: str ): Se ingresa un idioma (como están escritos en el dataset, no hay que traducirlos!). Debe devolver la cantidad de películas producidas en ese idioma.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en idioma

In [367]:
#df_pruebas['original_language'] = df_pruebas['original_language'].fillna('')

def buscar_peliculas_por_idioma(Idioma):
    # Filtrar el DataFrame por el idioma dado
    peliculas_filtradas = df_pruebas[df_pruebas['original_language'].apply(lambda x: pd.notna(x) and Idioma in x)]
    # Obtener una lista de los nombres de las películas filtradas
    nombres_peliculas = peliculas_filtradas['original_language'].count()
    return nombres_peliculas

buscar_peliculas_por_idioma('en')



32192

def peliculas_duracion( Pelicula: str ): Se ingresa una pelicula. Debe devolver la la duracion y el año.
                    Ejemplo de retorno: X . Duración: x. Año: xx

In [309]:
def obtener_info_pelicula(nombre_pelicula):
    # Filtrar el DataFrame por el nombre de la película
    filtro = df_pruebas['title'] == nombre_pelicula
    resultado = df_pruebas.loc[filtro, ['release_date', 'runtime']]

    # Obtener el año de estreno y la duración
    año_estreno = resultado['release_date'].values[0].split('-')[0]
    duracion = resultado['runtime'].values[0]

    # Retornar el año de estreno y la duración como diccionario
    return {'año_estreno': año_estreno, 'duracion': duracion}

info_pelicula = obtener_info_pelicula('Balto')
print("Año de estreno:", info_pelicula['año_estreno'])
print("Duración:", info_pelicula['duracion'])

Año de estreno: 1995
Duración: 78.0


def franquicia( Franquicia: str ): Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio
                    Ejemplo de retorno: La franquicia X posee X peliculas, una ganancia total de x y una ganancia

In [311]:
def obtener_estadisticas_franquicia(franquicia):
    # Filtrar el DataFrame por la franquicia dada
    peliculas_franquicia = df_pruebas[df_pruebas['belongs_to_collection'] == franquicia]

    # Calcular el número de películas
    num_peliculas = len(peliculas_franquicia)

    # Calcular el monto total de revenue
    monto_total_revenue = peliculas_franquicia['revenue'].sum()

    # Calcular el promedio de revenue
    promedio_revenue = peliculas_franquicia['revenue'].mean()

    # Retornar las estadísticas como un diccionario
    return {'numero_peliculas': num_peliculas, 'monto_total_revenue': monto_total_revenue, 'promedio_revenue': promedio_revenue}

estadisticas = obtener_estadisticas_franquicia('James Bond Collection')
print("Número de películas:", estadisticas['numero_peliculas'])
print("Monto total de ganancias:", estadisticas['monto_total_revenue'])
print("Promedio de ganancias:", estadisticas['promedio_revenue'])

Número de películas: 26
Monto total de ganancias: 7106970239.0
Promedio de ganancias: 273345009.1923077


def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!), retornando la cantidad de peliculas producidas en el mismo.
                    Ejemplo de retorno: Se produjeron X películas en el país X

In [320]:
def contar_peliculas_por_pais(pais):
    # Filtrar el DataFrame por el país dado, excluyendo valores NaN
    peliculas_pais = df_pruebas[df_pruebas['production_countries'].apply(lambda x: pd.notna(x) and pais in x)]

    # Contar la cantidad de películas
    cantidad_peliculas = len(peliculas_pais)

    return cantidad_peliculas


cantidad_peliculas = contar_peliculas_por_pais('Mexico')
print("Cantidad de películas:", cantidad_peliculas)

Cantidad de películas: 329


def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.
                    Ejemplo de retorno: La productora X ha tenido un revenue de x

In [330]:
def obtener_estadisticas_productora(productora):
    # Filtrar el DataFrame por la productora dada
    peliculas_productora = df_pruebas[df_pruebas['production_companies'].apply(lambda x: pd.notna(x) and productora in x)]

    # Contar el número de películas
    num_peliculas = len(peliculas_productora)

    # Calcular el revenue total
    revenue_total = peliculas_productora['revenue'].sum()

    return {'numero_peliculas': num_peliculas, 'revenue_total': revenue_total}



estadisticas_productora = obtener_estadisticas_productora('Pixar Animation Studios')
print("Número de películas:", estadisticas_productora['numero_peliculas'])
print("Monto total de ganancias:", estadisticas_productora['revenue_total'])



Número de películas: 52
Monto total de ganancias: 11188533734.0


def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma, en formato lista.

In [337]:
def obtener_estadisticas_director(nombre_director):

    peliculas_director = df_pruebas[df_pruebas['director'].apply(lambda x: pd.notna(x) and nombre_director in x)]

    # Calcular la suma de returns
    suma_returns = peliculas_director['return'].sum()

    # Obtener la lista de películas y sus detalles
    peliculas = peliculas_director[['title', 'release_date', 'return', 'budget', 'revenue']].values.tolist()

    return suma_returns, peliculas

nombre_director = "Robert Rodriguez"
suma_returns, peliculas = obtener_estadisticas_director(nombre_director)
print("Suma de returns:", suma_returns)
print("Películas en las que trabajó:")
for pelicula in peliculas:
    print(pelicula)

Suma de returns: 46.647748199526596
Películas en las que trabajó:
['Four Rooms', '1995-12-09', 1.075, 4000000, 4300000.0]
['From Dusk Till Dawn', '1996-01-19', 1.359821894736842, 19000000, 25836616.0]
['Desperado', '1995-08-25', 3.6293492857142855, 7000000, 25405445.0]
['The Faculty', '1998-12-25', 2.6855547333333334, 15000000, 40283321.0]
['El Mariachi', '1992-09-04', 9.276909090909092, 220000, 2040920.0]
['Spy Kids', '2001-03-18', 4.226690857142857, 35000000, 147934180.0]
['Spy Kids 2: The Island of Lost Dreams', '2002-08-07', 3.1506146842105265, 38000000, 119723358.0]
['Spy Kids 3-D: Game Over', '2003-07-25', 5.184525842105264, 38000000, 197011982.0]
['Once Upon a Time in Mexico', '2003-09-11', 3.3857097241379317, 29000000, 98185582.0]
['Sin City', '2005-04-01', 3.9683455, 40000000, 158733820.0]
['The Adventures of Sharkboy and Lavagirl', '2005-06-10', 1.38851932, 50000000, 69425966.0]
['Planet Terror', '2007-04-06', 0.0, 0, 0.0]
['Shorts', '2009-08-21', 0.7243127, 40000000, 2897250